In [1]:
from IPython.display import Image
from IPython.display import HTML, display
from PIL import Image, ImageFilter
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from statsmodels.graphics.mosaicplot import mosaic
import warnings 
warnings.filterwarnings("ignore")
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import plot_tree
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D
from keras.layers import Dense, Activation, Dropout, Flatten
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler 
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
import tensorflow as tf
from tensorflow.python.client import device_lib
import keras_tuner as kt


#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:

trainpt = pd.read_csv("train.csv")

In [2]:

import tensorflow as tf
from tensorflow.python.client import device_lib


print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("Memoria GPU configurada para crecimiento dinámico.")
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


GPUs disponibles: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memoria GPU configurada para crecimiento dinámico.
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6685683657292318830
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5713690624
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10282146362433573378
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


In [3]:
#prepara los datos para que sea más fácil acceder a las imágenes utilizando las rutas generadas.
def train_id_to_path(x):
    return 'train/' + x + ".jpg"
def test_id_to_path(x):
    return 'test/' + x + ".jpg"
traint = pd.read_csv('train.csv')
train11=traint[["Id","Pawpularity"]]
testt = pd.read_csv('test.csv')
test11 = testt[["Id"]] 
train11["img_path"] = train11["Id"].apply(train_id_to_path)
test11["img_path"] = test11["Id"].apply(test_id_to_path)

In [4]:
# La función devuelve un tensor que representa la imagen
# Normalizada (valores entre 0 y 1).
# Redimensionada a 128x128 píxeles.
# Con 3 canales de color (RGB).
image_height = 128
image_width = 128
def path_to_eagertensor(image_path):
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (image_height, image_width))
    return image

In [5]:
# Organizar datos en un formato estándar: Al convertir 
# la lista de tensores en un array de NumPy,
# se asegura que las imágenes se puedan procesar 
# eficientemente en lotes para el entrenamiento del modelo.

X1 = []
for img in train11['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X1.append(new_img_tensor)
    
print(type(X1),len(X1))
X1 = np.array(X1)
print(type(X1),X1.shape)

<class 'list'> 9912
<class 'numpy.ndarray'> (9912, 128, 128, 3)


In [6]:
#realiuzamos lo mismo para el conjunto de test
X1_sub = []
for img in test11['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X1_sub.append(new_img_tensor)
    
print(type(X1_sub),len(X1_sub))
X1_sub = np.array(X1_sub)
print(type(X1_sub),X1_sub.shape)
y1=train11["Pawpularity"]

<class 'list'> 8
<class 'numpy.ndarray'> (8, 128, 128, 3)


In [7]:
x_train, x_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(7929, 128, 128, 3) (1983, 128, 128, 3) (7929,) (1983,)


In [8]:
inputs = tf.keras.Input(shape=(image_height,image_width,3))
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2),padding='valid',activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = inputs, outputs = output)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 61, 61, 16)        2368      
                                                                 
 conv2d_1 (Conv2D)           (None, 31, 31, 32)        4640      
                                                                 
 batch_normalization (BatchN  (None, 31, 31, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 31, 31, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 31, 31, 64)       256       
 hNormalization)                                             

In [10]:
model.compile(optimizer='Adam', loss='mse', metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"]) 

In [11]:
# ImageDataGenerator es una clase de Keras que permite realizar transformaciones
# en las imágenes para crear nuevas versiones (aumentación) en tiempo real durante
# el entrenamiento. Esto es útil para evitar el sobreajuste y aumentar la diversidad
# de los datos de entrenamiento.

data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")

In [12]:
history = model.fit(
    data_augmentation.flow(x_train,y_train,batch_size=32),
    validation_data = (x_test,y_test),
    epochs = 3
)

Epoch 1/3
248/248 [==============================] - 25s 78ms/step - loss: 507.6636 - rmse: 22.5314 - mae: 16.8393 - mape: 81.5462 - val_loss: 460.2799 - val_rmse: 21.4541 - val_mae: 16.3842 - val_mape: 79.9077
Epoch 2/3
248/248 [==============================] - 19s 75ms/step - loss: 480.3781 - rmse: 21.9175 - mae: 16.4458 - mape: 80.5542 - val_loss: 474.0730 - val_rmse: 21.7732 - val_mae: 15.2639 - val_mape: 65.2966
Epoch 3/3
248/248 [==============================] - 19s 75ms/step - loss: 474.3529 - rmse: 21.7796 - mae: 16.2891 - mape: 79.8476 - val_loss: 453.2065 - val_rmse: 21.2886 - val_mae: 15.1357 - val_mape: 67.5263


In [13]:

# Crear el objeto de búsqueda con un objetivo explícito

def build_model(hp):
    inputs = tf.keras.Input(shape=(image_height, image_width, 3))
    x = inputs

    # Elegir dinámicamente el tamaño del kernel
    kernel_size = hp.Choice('kernel_size', values=[3, 5, 7])  # Solo enteros

    x = tf.keras.layers.Conv2D(
        filters=hp.Choice('filters_layer_1', values=[16, 32, 64]),
        kernel_size=(kernel_size, kernel_size),  # Crear la tupla aquí
        strides=(2, 2),
        padding='valid',
        activation='relu'
    )(x)

    x = tf.keras.layers.Conv2D(
        filters=hp.Choice('filters_layer_2', values=[32, 64, 128]),
        kernel_size=(kernel_size, kernel_size),
        strides=(2, 2),
        padding='same',
        activation='relu'
    )(x)

    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(
        filters=hp.Choice('filters_layer_3', values=[64, 128, 256]),
        kernel_size=(kernel_size, kernel_size),
        padding='same',
        activation='relu'
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(
        units=hp.Choice('dense_units', values=[256, 512, 1024]),
        activation='relu'
    )(x)
    x = tf.keras.layers.Dropout(hp.Choice('dropout_rate', values=[0.3, 0.5, 0.7]))(x)
    output = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='mse',
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
    )
    return model



In [14]:

# Crear el objeto de búsqueda
tuner = kt.Hyperband(
    build_model,
    objective=kt.Objective("val_rmse", direction="min"),  # Especificar la métrica personalizada
    max_epochs=10,
    factor=3,
    directory='hyperband_dir',
    project_name='pawpularity_optimization'
)

# Llamada para buscar mejores hiperparámetros
tuner.search(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=32
)

# Mejor configuración encontrada
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Mejores Hiperparámetros:", best_hps.values)


Trial 28 Complete [00h 01m 29s]
val_rmse: 21.58303451538086

Best val_rmse So Far: 20.392410278320312
Total elapsed time: 10h 15m 59s
Mejores Hiperparámetros: {'kernel_size': 5, 'filters_layer_1': 32, 'filters_layer_2': 64, 'filters_layer_3': 64, 'dense_units': 1024, 'dropout_rate': 0.5, 'learning_rate': 0.0001, 'tuner/epochs': 4, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}


In [15]:
def build_model_v2(hp):
    inputs = tf.keras.Input(shape=(image_height, image_width, 3))
    x = inputs

    # Primera capa convolucional
    x = tf.keras.layers.Conv2D(
        filters=hp.Choice('filters_layer_1', values=[32, 64, 128]),
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid',
        activation='relu'
    )(x)

    # Segunda capa convolucional
    x = tf.keras.layers.Conv2D(
        filters=hp.Choice('filters_layer_2', values=[64, 128, 256]),
        kernel_size=(3, 3),
        padding='same',
        activation='relu'
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Tercera capa convolucional
    x = tf.keras.layers.Conv2D(
        filters=hp.Choice('filters_layer_3', values=[128, 256, 512]),
        kernel_size=(3, 3),
        padding='same',
        activation='relu'
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Regularización y capa densa
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(
        units=hp.Choice('dense_units', values=[128, 256, 512]),
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(hp.Choice('l2_rate', values=[1e-4, 1e-3]))
    )(x)
    x = tf.keras.layers.Dropout(hp.Choice('dropout_rate', values=[0.1, 0.3, 0.5]))(x)

    # Salida con activación sigmoide
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    output = output * 100  # Escalar a rango 0-100

    # Compilar modelo
    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])
        ),
        loss='mse',
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
    )
    return model


In [16]:

# Crear el objeto de búsqueda de hiperparámetros
tuner = kt.Hyperband(
    build_model_v2,
    objective=kt.Objective("val_rmse", direction="min"),  # Especificar la métrica personalizada
    max_epochs=10,
    factor=3,
    directory='hyperband_dir_v2',
    project_name='pawpularity_optimization_v2'
)

# Ejecutar la búsqueda de los mejores hiperparámetros
tuner.search(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=32
)

# Mostrar la mejor configuración encontrada
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Mejores Hiperparámetros:", best_hps.values)

Trial 30 Complete [00h 01m 39s]
val_rmse: 43.44029235839844

Best val_rmse So Far: 20.930692672729492
Total elapsed time: 00h 31m 18s
Mejores Hiperparámetros: {'filters_layer_1': 32, 'filters_layer_2': 256, 'filters_layer_3': 128, 'dense_units': 128, 'l2_rate': 0.001, 'dropout_rate': 0.3, 'learning_rate': 0.0001, 'tuner/epochs': 4, 'tuner/initial_epoch': 2, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0000'}


Progreso Inicial:

El modelo está aprendiendo, como lo muestra la reducción significativa del loss, mae, y rmse entre las épocas 1 y 2.
Estabilidad:

La estabilidad de las métricas (val_loss y val_rmse) entre las épocas 2 y 3 indica que el modelo ha alcanzado un punto de convergencia, aunque no óptimo.
Error Relativo Alto:

El mape y el rmse sugieren que el modelo aún no predice con alta precisión. Un rmse cercano a 21 unidades sigue siendo significativo, dado que la escala de "Pawpularity" es de 0 a 100.
Posible Sobreajuste:

El incremento en val_loss y val_rmse en la tercera época podría ser un signo de sobreajuste temprano.

In [17]:
def convert2onehot_4(labels):
    result = []
    for label in labels:
#         print(label)
        if label == '0':
#             print("label 1")
            result.append([1, 0, 0, 0])
        elif label == '1':
            result.append([0,1,0,0])
        elif label == '2':
            result.append([0,0,1,0])
        else:
#             print("label 0")
            result.append([0,0,0,1])
    return np.array(result)
x_train_array = np.array(x_train)
x_test_array = np.array(x_test)
y_train_onehot = convert2onehot_4(y_train)
y_test_onehot = convert2onehot_4(y_test)

In [18]:
inputs1 = tf.keras.Input(shape=(image_height,image_width,3))
x1 = inputs1

x1 = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2),padding='valid',activation = 'relu')(x1)
x1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)


x1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.Dropout(0.25)(x1)

x1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x1)
x1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.Dropout(0.25)(x1)

x1 = tf.keras.layers.Flatten()(x1)
x1 = tf.keras.layers.Dense(512, activation = "relu")(x1)
x1 = tf.keras.layers.Dropout(0.5)(x1)

output1 = tf.keras.layers.Dense(4)(x1)

model1 = tf.keras.Model(inputs = inputs1, outputs = output1)

In [19]:
model1.compile(optimizer='Adam', loss='mse', metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"]) 
print(x_train_array.shape, y_train_onehot.shape,x_test_array.shape, y_test_onehot.shape)

(7929, 128, 128, 3) (7929, 4) (1983, 128, 128, 3) (1983, 4)


In [20]:
model1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 16)        2368      
                                                                 
 conv2d_4 (Conv2D)           (None, 31, 31, 32)        4640      
                                                                 
 batch_normalization_2 (Batc  (None, 31, 31, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, 31, 31, 64)        18496     
                                                                 
 batch_normalization_3 (Batc  (None, 31, 31, 64)       256       
 hNormalization)                                           

In [21]:
model1.fit(x_train_array, y_train_onehot, batch_size=16, epochs=3, validation_data=(x_test_array, y_test_onehot))

Epoch 1/3
496/496 [==============================] - 8s 14ms/step - loss: 0.2197 - rmse: 0.4687 - mae: 0.1806 - mape: 85701296.0000 - val_loss: 0.1545 - val_rmse: 0.3931 - val_mae: 0.1748 - val_mape: 99758616.0000
Epoch 2/3
496/496 [==============================] - 6s 12ms/step - loss: 0.0163 - rmse: 0.1278 - mae: 0.0557 - mape: 4506649.5000 - val_loss: 0.0656 - val_rmse: 0.2561 - val_mae: 0.0723 - val_mape: 31532024.0000
Epoch 3/3
496/496 [==============================] - 7s 13ms/step - loss: 0.0061 - rmse: 0.0779 - mae: 0.0330 - mape: 1971696.5000 - val_loss: 0.0456 - val_rmse: 0.2135 - val_mae: 0.0403 - val_mape: 25925312.0000


In [22]:
cnn_pred = model.predict(X1_sub)
print(X1_sub.shape, type(X1_sub))
print(cnn_pred.shape, type(cnn_pred))

1/1 [==============================] - 0s 254ms/step
(8, 128, 128, 3) <class 'numpy.ndarray'>
(8, 1) <class 'numpy.ndarray'>


In [23]:
cnn = pd.DataFrame()
cnn['Id'] = test11['Id']
cnn['Pawpularity'] = cnn_pred
cnn.to_csv('submission.csv',index=False)

In [24]:
cnn.head(10)

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,37.240929
1,43a2262d7738e3d420d453815151079e,37.549671
2,4e429cead1848a298432a0acad014c9d,37.808235
3,80bc3ccafcc51b66303c2c263aa38486,37.142014
4,8f49844c382931444e68dffbe20228f4,38.097736
5,b03f7041962238a7c9d6537e22f9b017,37.510044
6,c978013571258ed6d4637f6e8cc9d6a3,37.887524
7,e0de453c1bffc20c22b072b34b54e50f,37.140045


In [25]:
cnn_pred1 = model1.predict(X1_sub)
print(X1_sub.shape, type(X1_sub))
print(cnn_pred1.shape, type(cnn_pred1))

1/1 [==============================] - 0s 113ms/step
(8, 128, 128, 3) <class 'numpy.ndarray'>
(8, 4) <class 'numpy.ndarray'>


In [26]:
cnn1=pd.DataFrame(cnn_pred1)
cnn1

,0,1,2,3
0,0.000381,0.001802,0.000563,0.947257
1,0.000322,0.001794,0.000645,0.950817
2,0.000448,0.002077,0.000727,0.957185
3,0.000399,0.001804,0.000506,0.961188
4,0.000390,0.001895,0.000601,0.952846
5,0.000399,0.001937,0.000695,0.948315
6,0.000360,0.001661,0.000460,0.943971
7,0.000425,0.001906,0.000578,0.956783
